In [16]:
import random as rnd
import math

In [17]:
def generateGridNetwork(n,d):
    edges = []
    for i in range(n):
        for j in range(n):
            # nodo orizzontale
            if j != n - 1:
                edges.append([j + i * n, j + i * n + 1])
            # nodo verticale
            if i != n - 1:
                edges.append([j + i * n, j + (i + 1) * n])
    tmp = edges.copy()
    for i in tmp:
        edges.append([i[1], i[0]])
    buf = "param: E: d:=\n"
    for i in edges:
        buf += " " + str(i[0]) + " " + str(i[1]) + " " + str(d)
        if (i == edges[-1]):
            buf += ";"
        buf += "\n"

    #for i in range(n * n):
    #    buf += " " + str(i)
    #buf += " :=\n"
    #matrix = [["-" for i in range(n * n)] for j in range(n * n)]
    #for edge in edges:
    #    matrix[edge[0]][edge[1]] = "+"
    #for i in range(n * n):
    #    if i == n * n - 1:
    #        buf += str(i) + " " + " ".join(matrix[i]) + " ;\n"
    #    else:
    #        buf += str(i) + " " + " ".join(matrix[i]) + "\n"
    return (buf,edges)

In [18]:
def generateFlights(nf, nn, seed):
    rnd.seed(seed)
    buff = "param s:=\n"
    s = [-1] * nf
    e = [-1] * nf
    for i in range(nf):
        s[i] = rnd.randint(0, nn - 1)
        e[i] = rnd.randint(0, nn - 1)
        while e[i] == s[i]:
            e[i] = rnd.randint(0, nn - 1)
        if i == nf - 1:
            buff += str(i) + " " + str(s[i]) + " ;\n"
        else:
            buff += str(i) + " " + str(s[i]) + "\n"
    buff += "param e:=\n"
    for i in range(nf):
        if i == nf - 1:
            buff += str(i) + " " + str(e[i]) + " ;\n"
        else:
            buff += str(i) + " " + str(e[i]) + "\n"
    return buff

In [19]:
def generateDistance(matrix, d):
    buf = matrix
    buf = buf.replace("set", "param")
    buf = buf.replace("E", "d")
    buf = buf.replace("+", str(d))
    buf = buf.replace("-", ".")
    return buf

In [20]:
def generateSpeed(nn, nf, vMin,vMax,edges):
    buf = "param v_min:=\n"
    for i in range(nf):
        for (j,k) in edges:
            buf+= str(i) + " " + str(j) + " " + str(k) + " " + str(vMin) + "\t\t"
        if (i == nf-1):
            buf+= " ;"
        buf+= "\n"

    buf += "param v_max:=\n"
    for i in range(nf):
        for (j,k) in edges:
            buf+= str(i) + " " + str(j) + " " + str(k) + " " + str(vMax) + "\t\t"
        if (i == nf-1):
            buf+= " ;"
        buf+= "\n"

    return buf

In [21]:
def generateAngle(edges,angle):
    buf = "param angleM:=\n"
    for (x1,x) in edges:
        for (x2,x_inner) in edges:
            if x1!=x2 and x == x_inner:
                buf += str(x) + " " + str(x1) + " " + str(x2) + " " + str(angle) + "\t\t"
        if [x1,x] == edges[-1]:
            buf += ";"
        buf += "\n"

    buf += "param angleP:=\n"
    for (x,x1) in edges:
        for (x_inner,x2) in edges:
            if x1!=x2 and x == x_inner:
                buf += str(x) + " " + str(x1) + " " + str(x2) + " " + str(angle) + "\t\t"
        if [x,x1] == edges[-1]:
            buf += ";"
        buf += "\n"

    buf += "param anglePM:=\n"
    for (x,x1) in edges:
        for (x2,x_inner) in edges:
            if x1!=x2 and x == x_inner:
                buf += str(x) + " " + str(x1) + " " + str(x2) + " " + str(angle) + "\t\t"
        buf += "\n"
        if [x,x1] == edges[-1]:
            buf += ";\n"

    return buf

In [22]:
def generateTime(nn,nf,distanceTime=2):
    buf = "param t_hat_ear:"
    for i in range(nn):
        buf += " " + str(i)
    buf += " :=\n"
    for i in range(nf):
        buf += str(i)
        for j in range(nn):
            if j == nn - 1:
                if i == nf - 1:
                    buf += " 0 ;\n"
                else:
                    buf += " 0\n"
            else:
                buf += " 0"
    buf += "param t_hat_lat:"
    for i in range(nn):
        buf += " " + str(i)
    buf += " :=\n"
    for i in range(nf):
        buf += str(i)
        for j in range(nn):
            if j == nn - 1:
                if i == nf - 1:
                    buf += " " + str(distanceTime) + " ;\n"
                else:
                    buf += " " + str(distanceTime) + "\n"
            else:
                buf += " " + str(distanceTime)
    return buf

# Grid creation of graphs

In [23]:
def generateGridInstance(nf, nn, seed, d=10, v_min=1,v_max=2, D=2, distanceTime=5):
    angle = 1
    buff = "param nf:= " + str(nf) + " ;\n"
    buff += "param nn:= " + str(nn * nn) + " ;\n"
    # buff+="set F:=0.."+ str(nf-1)+";\n"
    # buff+="set V:=0.."+ str(nn*nn-1)+";\n"
    (temp,edges) = generateGridNetwork(nn,d)
    buff += temp
    buff += generateFlights(nf, nn * nn, seed)
    #buff += generateDistance(temp, d)
    buff += generateSpeed(nn * nn, nf, v_min, v_max, edges)
    buff += generateAngle(edges,angle)
    buff += "param D:=" + str(D) + " ;\n"
    buff += generateTime(nn*nn,nf,distanceTime)
    filename = "data/nn" + str(nn)+ "nf" +str(nf)+"/grid" + str(seed) + ".dat"
    file = open(filename, "w")
    file.write(buff)
    file.close()
    return buff

# Metroplex graph creation

In [24]:
def generateMetroplexNetwork(n,d):
    edges = []
    for i in range(n):
        for j in range(n):
            if j != n - 1:
                edges.append([j + i * n, j + i * n + 1])
            if i != n - 1:
                edges.append([j + i * n, j + (i + 1) * n])

    # creo seconda griglia prendendo la prima e aggiungendo n*n
    tmp = []
    for [i, j] in edges:  # second grids
        tmp.append([i + n * n, j + n * n])
    edges += tmp
    tmp = edges.copy()
    for i in tmp:
        edges.append([i[1], i[0]])
    
    # primo ultima riga e prima colonna
    edges += [[n * (n - 1), n * n]]  # connection between grids
    # primo ultima riga e ultima colonna
    edges += [[n * (n+1) - 1,n * (n - 1) + n - 1]]

    buf = "param: E: d:=\n"
    for i in edges:
        buf += " " + str(i[0]) + " " + str(i[1]) + " "+str(d)
        if (i == edges[-1]):
            buf += ";"
        buf += "\n"
    #for i in range(2 * n * n):
    #    buf += "\t" + str(i)
    #buf += " :=\n"
    #matrix = [["-" for i in range(2 * n * n)] for j in range(2 * n * n)]
    #for edge in edges:
    #    matrix[edge[0]][edge[1]] = "+"
    #    matrix[edge[1]][edge[0]] = "+"
    ##remove duplicate arcs between grids
    #matrix[n * n][n * (n - 1)] = "-"
    #matrix[n * n - 1][n * n + n - 1] = "-"
    #for i in range(2 * n * n):
    #    if i == 2 * n * n - 1:
    #        buf += str(i) + "\t" + "\t".join(matrix[i]) + " ;\n"
    #    else:
    #        buf += str(i) + "\t" + "\t".join(matrix[i]) + "\n"
    return (buf, edges)

In [25]:
def generateMetroplexInstance(nf, nn, seed,d=10,vMin=1,vMax=2,D=2,distanceTime=5):
    angle = 1
    buff = "param nf:= " + str(nf) + " ;\n"
    buff += "param nn:= " + str(nn * nn * 2) + " ;\n"
    # buff+="set F:=0.."+ str(nf-1)+";\n"
    # buff+="set V:=0.."+ str(nn*nn-1)+";\n"
    temp,edges = generateMetroplexNetwork(nn,d)
    buff += temp
    buff += generateFlights(nf, 2 * nn * nn, seed)
    #buff += generateDistance(temp, d)
    buff += generateSpeed(2 * nn * nn, nf, vMin,vMax,edges)
    buff += generateAngle(edges, angle)
    buff += "param D:=" + str(D) + " ;\n"
    buff += generateTime(2 * nn * nn,nf,distanceTime)
    filename = "data/nn" + str(nn)+ "nf" +str(nf)+"/metroplex" + str(seed) + ".dat"
    file = open(filename, "w")
    file.write(buff)
    file.close()
    return buff

# Airport graph creation

In [26]:
def generateAirportNetwork(n,d):
    edges = []
    for i in range(n):
        edges.append([i, n])
        if i!= n-1:
            edges.append([i, i + 1])
        else:
            edges.append([i, 0])
    tmp = edges.copy()
    for i in tmp:
        edges.append([i[1], i[0]])
    buf = "param: E: d:=\n"
    for i in edges:
        buf += " " + str(i[0]) + " " + str(i[1]) + " " + str(d) 
        if (i == edges[-1]):
            buf += ";"
        buf += "\n"
    #for i in range(n + 1):
    #    buf += " " + str(i)
    #buf += " :=\n"
    #matrix = [["-" for i in range(n + 1)] for j in range(n + 1)]
    #for edge in edges:
    #    matrix[edge[0]][edge[1]] = "+"
    #    matrix[edge[1]][edge[0]] = "+"
    #for i in range(n + 1):
    #    if i == n:
    #        buf += str(i) + " " + " ".join(matrix[i]) + " ;\n"
    #    else:
    #        buf += str(i) + " " + " ".join(matrix[i]) + "\n"
    return (buf,edges)


print(generateAirportNetwork(10,10))

('param: E: d:=\n 0 10 10\n 0 1 10\n 1 10 10\n 1 2 10\n 2 10 10\n 2 3 10\n 3 10 10\n 3 4 10\n 4 10 10\n 4 5 10\n 5 10 10\n 5 6 10\n 6 10 10\n 6 7 10\n 7 10 10\n 7 8 10\n 8 10 10\n 8 9 10\n 9 10 10\n 9 0 10\n 10 0 10\n 1 0 10\n 10 1 10\n 2 1 10\n 10 2 10\n 3 2 10\n 10 3 10\n 4 3 10\n 10 4 10\n 5 4 10\n 10 5 10\n 6 5 10\n 10 6 10\n 7 6 10\n 10 7 10\n 8 7 10\n 10 8 10\n 9 8 10\n 10 9 10\n 0 9 10;\n', [[0, 10], [0, 1], [1, 10], [1, 2], [2, 10], [2, 3], [3, 10], [3, 4], [4, 10], [4, 5], [5, 10], [5, 6], [6, 10], [6, 7], [7, 10], [7, 8], [8, 10], [8, 9], [9, 10], [9, 0], [10, 0], [1, 0], [10, 1], [2, 1], [10, 2], [3, 2], [10, 3], [4, 3], [10, 4], [5, 4], [10, 5], [6, 5], [10, 6], [7, 6], [10, 7], [8, 7], [10, 8], [9, 8], [10, 9], [0, 9]])


In [27]:
def calculateAngle(nn, x,y,z):
    if x==nn-1 : #x è il nodo centrale
        return math.sin(math.radians(360/(nn-1)*min(abs(y-z),nn-1-abs(y-z)))) if (360/(nn-1)*min(abs(y-z),nn-1-abs(y-z)))<90 else 1
    if y==nn-1 or z==nn-1: # x1 o x2 è il nodo centrale
        return math.sin(math.radians((180-360/(nn-1))/2)) if ((180-360/(nn-1))/2) <90 else 1
    return math.sin(math.radians(180-360/(nn-1))) if (180-360/(nn-1)) <90 else 1


In [28]:
def generateAirportAngle(nn, edges):
    buf = "param angleM:=\n"
    for (x1,x) in edges:
        for (x2,x_inner) in edges:
            if (x1!=x2 and x==x_inner):
                buf += str(x) + " " + str(x1) + " " + str(x2) + " " + str(round(1/calculateAngle(nn,x,x1,x2),6)) + "\t\t"
        if [x1,x] == edges[-1]:
            buf += ";"
        buf += "\n"

    buf += "param angleP:=\n"
    for (x,x1) in edges:
        for (x_inner,x2) in edges:
            if (x1!=x2 and x==x_inner):
                buf += str(x) + " " + str(x1) + " " + str(x2) + " " + str(round(1/calculateAngle(nn,x,x1,x2),6)) + "\t\t"
        if [x,x1] == edges[-1]:
            buf += ";"
        buf += "\n"

    buf += "param anglePM:=\n"
    for (x,x1) in edges:
        for (x2,x_inner) in edges:
            if (x1!=x2 and x==x_inner):
                buf += str(x) + " " + str(x1) + " " + str(x2) + " " + str(round(1/calculateAngle(nn,x,x1,x2),6)) + "\t\t"
        if [x,x1] == edges[-1]:
            buf += ";"
        buf += "\n"

    return buf

In [29]:
def generateAirportInstance(nf, nn, seed,d=10, vMin=1,vMax=2,D=2,distanceTime=5, stringnn=""):
    buff = "param nf:= " + str(nf) + " ;\n"
    buff += "param nn:= " + str(nn + 1) + " ;\n"
    # buff+="set F:=0.."+ str(nf-1)+";\n"
    # buff+="set V:=0.."+ str(nn*nn-1)+";\n"
    temp, edges= generateAirportNetwork(nn,d)
    buff += temp
    buff += generateFlights(nf, nn + 1, seed)
    #buff += generateDistance(temp, d)
    buff += generateSpeed(nn + 1, nf, vMin,vMax,edges)
    buff += generateAirportAngle(nn + 1, edges)
    buff += "param D:=" + str(D) + " ;\n"
    buff += generateTime(nn+1,nf,distanceTime)
    filename = "data/nn" + stringnn+ "nf" +str(nf)+"/airport" + str(seed) + ".dat"
    file = open(filename, "w")
    file.write(buff)
    file.close()
    return buff

# Generation of graphs

In [30]:
for i in range (3,7,1):
    nodes = i
    flights = (i-2)*5
    print("starting nn ",nodes," nf",flights)
    for j in range(20):
        generateGridInstance(flights, nodes, j, distanceTime=2)
        generateMetroplexInstance(flights, nodes, j,distanceTime=2)
        generateAirportInstance(flights, nodes * nodes - 1, j,distanceTime=2,stringnn=str(nodes))
        print("instace ",j," generated")
nodes = 10
flights = 100
print("starting 100,100")
for j in range(20):
    generateGridInstance(flights, nodes, j, distanceTime=2)
    generateMetroplexInstance(flights, nodes, j,distanceTime=2)
    generateAirportInstance(flights, nodes * nodes - 1, j,distanceTime=2,stringnn=str(nodes))
    print("instace ",j," generated")


starting nn  3  nf 5
instace  0  generated
instace  1  generated
instace  2  generated
instace  3  generated
instace  4  generated
instace  5  generated
instace  6  generated
instace  7  generated
instace  8  generated
instace  9  generated
instace  10  generated
instace  11  generated
instace  12  generated
instace  13  generated
instace  14  generated
instace  15  generated
instace  16  generated
instace  17  generated
instace  18  generated
instace  19  generated
starting nn  4  nf 10
instace  0  generated
instace  1  generated
instace  2  generated
instace  3  generated
instace  4  generated
instace  5  generated
instace  6  generated
instace  7  generated
instace  8  generated
instace  9  generated
instace  10  generated
instace  11  generated
instace  12  generated
instace  13  generated
instace  14  generated
instace  15  generated
instace  16  generated
instace  17  generated
instace  18  generated
instace  19  generated
starting nn  5  nf 15
instace  0  generated
instace  1  g

instace  7  generated
instace  8  generated
instace  9  generated
instace  10  generated
instace  11  generated
instace  12  generated
instace  13  generated
instace  14  generated
instace  15  generated
instace  16  generated
instace  17  generated
instace  18  generated
instace  19  generated
starting nn  6  nf 20
instace  0  generated
instace  1  generated
instace  2  generated
instace  3  generated
instace  4  generated
instace  5  generated
instace  6  generated
instace  7  generated
instace  8  generated
instace  9  generated
instace  10  generated
instace  11  generated
instace  12  generated
instace  13  generated
instace  14  generated
instace  15  generated
instace  16  generated
instace  17  generated
instace  18  generated
instace  19  generated
starting 100,100
instace  0  generated
instace  1  generated
instace  2  generated
instace  3  generated
instace  4  generated
instace  5  generated
instace  6  generated
instace  7  generated
instace  8  generated
instace  9  gener